In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

## Disclaimer

The architecture of the following model is pureley fictous and its only purpose is to show you how to implement

- a custom Layer Block
- a call with a residual connection
- concatenation

Do not refer to this architecture for the homework!!!!!

In [3]:
# define arbitrary custom layer block which you can use like a layer in your model
# a layer block is basicall a subclass containing multiple layers
# to make it callable, inherit from tf.keras.layers.Layer

class CustomBlock(tf.keras.layers.Layer):
  
  def __init__(self, channels):
    super(CustomBlock, self).__init__()

    # define set of layers

    # be careful! with residual connections, you have to make sure that the dimensions from your input match the block output
    # therefore be careful what filter size youuse and choose sme padiing
    self.conv = tf.keras.layers.Conv2D(filters=channels, kernel_size=3, padding='SAME')
    # you can put multiple more layers here

  def call(self, x):

    # define call
    # add original input to block output
    y = self.conv(x)
    y = y + x

    return y


In [5]:
x = tf.ones((1,10,10,3))
block = CustomBlock(channels=3)
y = block(x)
print(y.shape)

(1, 10, 10, 3)


In [24]:
# define network
class CustomNet(tf.keras.Model):

  def __init__(self, num_blocks=2, channels_in_block=10):

    super(CustomNet, self).__init__()

    # you can define a set of layes to put in front of your custom blocks
    self.pre_block = [tf.keras.layers.Conv2D(filters=channels_in_block, kernel_size=3, padding='SAME')]
    self.pre_block.append(tf.keras.layers.BatchNormalization())

    # now you can use your self defined blocks much like layers
    self.blocks = []
    for _ in range(num_blocks):
        self.blocks.append(CustomBlock(channels=channels_in_block))
        # append more stuff if you want to alter the order in the call
        # eg Transition Layers

    # concatenation layer    
    self.concat = tf.keras.layers.Concatenate()
    
    # readout layers
    self.post_blocks = []
    self.post_blocks.append(tf.keras.layers.GlobalAveragePooling2D())
    self.post_blocks.append(tf.keras.layers.Dense(units=5))
   

    
  def call(self, x, training_flag=True):

      # pass input through pre_block layers
      # you can pass the training flag to every layer in tf
      for l in self.pre_block:
        x = l(x, training=training_flag)

      # concatenate blocks input and blocks output
      y = x
      for b in self.blocks:
        y = b(y)
      # the concatenation layer has to be called with a list of 
      # ! again fictional architecture !
      y = self.concat([y,x])
      
      # readout layers
      for b in self.post_blocks:
        y = b(y)

      # make sure prediction and target dimension match
      y = tf.expand_dims(y, axis=1)

      return y

In [25]:
def timing(start):
    now = time.time()
    time_per_training_step = now - start
    # compute duration of an epoch
    return round(time_per_training_step, 2)

In [26]:
# define arbitrary input and target
input = tf.zeros((4,100,100,3))
target = tf.ones((4,1,5))

In [27]:
tf.keras.backend.clear_session()
model = CustomNet()

loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

start = time.time()

with tf.GradientTape() as tape:
  prediction = model(input, training=True)
  loss = loss_function(target, prediction)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

print(f"The training step took {timing(start)} seconds")




The training step took 0.51 seconds


In [28]:
# how many parameters does our network have?
model.summary()

Model: "custom_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  280       
_________________________________________________________________
batch_normalization (BatchNo multiple                  40        
_________________________________________________________________
custom_block (CustomBlock)   multiple                  910       
_________________________________________________________________
custom_block_1 (CustomBlock) multiple                  910       
_________________________________________________________________
concatenate (Concatenate)    multiple                  0         
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1